In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import ndjson
import copy 

def gendata():    
    for obj in data:
        if 'event' in obj:
            if 'options' in obj['event']:
                if type(obj['event']['options']) is list:
                    obj['event']['arr_options'] =copy.deepcopy(obj['event']['options'])   
                    del obj['event']['options']
            
            if type(obj['event']) is str:                
                obj['str_event'] = obj.pop('event') 
            elif type(obj['event']) is list:
                obj['list_event'] = obj.pop('event')
                
            
        
        
        yield {
            "_index": "online_courses_full",
            "_type": "doc",
            "_source": obj
        }
        
settings={
    "index.mapping.total_fields.limit": 100000
}

es = Elasticsearch([{"host": "localhost", "port": 9200}])

if es.indices.exists("online_courses_full"):
    es.indices.delete("online_courses_full")

es.indices.create(index="online_courses_full", body=settings)

with open("data/encoded_logs.json", "r") as read_file:
    data = ndjson.load(read_file)
    bulk(es, gendata())

In [2]:
#96094